In [3]:
from IPython.display import clear_output
import numpy as np

Regressor class

In [4]:
class Regressor:

    def __init__(self, A_DIM, R_DIM):
        self.A_DIM = A_DIM
        self.R_DIM = R_DIM
        self.W = [np.random.random(
            (self.A_DIM, self.R_DIM)), np.random.random((self.R_DIM, self.A_DIM))]
        self.label_norm = 1

    def validate(self, a, b):
        self.variance, self.max_error, self.accuracy = self.metrics = self.compare_vector(
            a, b)

    def compare_vector(self, a, b):
        c = a - b
        c = np.squeeze(c)
        v = np.var(c)
        m = np.max(c)
        a = - np.log10(v) * 2
        return v, m, a

    def dropout_matrix(self, percent, shape):
        x = np.random.choice(2, shape, p=[1 - percent, percent])
        return x

    def fitUtil(self, training_data, training_label, alpha=0.0001, retain=1):

        hidden_layer = (training_data * self.dropout_matrix(retain,
                        training_data.shape[1])).dot(self.W[0])
        predicted_label = (
            hidden_layer * self.dropout_matrix(retain, hidden_layer.shape[1])).dot(self.W[1])

        error = training_label - predicted_label

        dW0 = - \
            training_data.T.dot(
                2 * error.dot(self.W[1].T)) / training_data.size
        dW1 = - hidden_layer.T.dot(2 * error) / training_data.size

        self.W[0] -= dW0 * alpha
        self.W[1] -= dW1 * alpha

    # Applies cross validation
    def fit(self, vecs, vecsy, epochs, **kwargs):

        vecs = np.array(np.split(vecs, 10))
        vecsy = np.array(np.split(vecsy, 10))

        for i in range(10):

            c_v_training_data, c_v_training_label = np.vstack(
                np.delete(vecs, i, axis=0)), np.vstack(np.delete(vecsy, i, axis=0))
            c_v_testing_data, c_v_testing_label = vecs[i], vecsy[i]

            for epoch in range(epochs // 10):
                self.fitUtil(c_v_training_data, c_v_training_label, **kwargs)

            self.validate(self.predict(c_v_testing_data), c_v_testing_label)
            return self.metrics

    def predict(self, data):
        return data.dot(self.W[0]).dot(self.W[1])

    def encode(self, data):
        return data.dot(self.W[0])

    def decode(self, data):
        return data.dot(self.W[1])

    def __str__(self):
        return '\n'.join(["Error variance " + str(self.variance),
                          "Error amplitude " + str(self.max_error),
                          "Accuracy " + str(self.accuracy),
                          "Weights" + str(self.W)])

    def __repr__(self):
        return '\n'.join(["Error variance " + str(self.variance),
                          "Error amplitude " + str(self.max_error),
                          "Accuracy " + str(self.accuracy),
                          "Weights" + str(self.W)])


In [5]:
class DataGenerator:
    
    def __init__(self, dim_a, dim_b, rotation_count = 0):
        self.actual_dim = dim_a
        self.reduced_dim = dim_b
        self.r = np.identity(self.actual_dim)
        for i in range(rotation_count):
            self.r = self.r.dot(self.set_rotational_matrix())


    def generate(self, data_count = 1000, noise = 0):
        
        vecs = (np.random.random((data_count, self.reduced_dim)) * 2) - 1
        vecs = np.append(vecs, np.random.random((data_count, self.actual_dim - self.reduced_dim)) * noise, axis = 1)
        vecs = vecs.dot(self.r)
        return vecs

    def set_rotational_matrix(self):
    
        theta = 2 * np.pi * np.random.random()
        axes = np.random.choice(self.actual_dim, 2, replace=False)

        i, j = axes[0], axes[1]

        rotational_matrix = np.identity(self.actual_dim)
        rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
        rotational_matrix[i][j] = np.sin(theta)
        rotational_matrix[j][i] = -np.sin(theta)

        return rotational_matrix

Tries reduction to each dimension below

In [21]:
def get_reduction_scores(training_data, testing_data, **kwargs):
    dim = testing_data.shape[1]
    reduced_dim = 1

    zero_reduction_score = 0

    for i in range(dim, 0, -1):
        
        print("Reduction progress", (1 - i/dim) * 100)
        regressor = Regressor(dim, i)
        regressor.fit(training_data, training_data, **kwargs)
        
        reduction_score = regressor.compare_vector(regressor.predict(testing_data), testing_data)[2]
        
        if i == dim:
            zero_reduction_score = reduction_score

        if reduction_score / zero_reduction_score * 100 > 15:
            reduced_dim = i
    
    kwargs['epochs'] *= 10
    regressor = Regressor(dim, reduced_dim)
    regressor.fit(training_data, training_data, **kwargs)   

    
    return regressor

Creating random data that is reducible to a random dimension

In [19]:
dim = np.random.randint(2, 20)
dim2 = np.random.randint(1, dim)
datagen = DataGenerator(dim, dim2, rotation_count = 2442)
training_data = datagen.generate(10000, noise=0)
testing_data = datagen.generate(3000, noise=0)
dim2, dim

(9, 11)

In [22]:
regressor = get_reduction_scores(training_data, testing_data, epochs = 10000, alpha = 1, retain = 1)


Reduction progress 0.0
Reduction progress 9.090909090909093
Reduction progress 18.181818181818176
Reduction progress 27.27272727272727
Reduction progress 36.36363636363637
Reduction progress 45.45454545454546
Reduction progress 54.54545454545454
Reduction progress 63.63636363636363
Reduction progress 72.72727272727273
Reduction progress 81.81818181818181
Reduction progress 90.9090909090909


In [23]:
v = datagen.generate(1, noise=0)
v, regressor.decode(regressor.encode(v))

(array([[-0.3131676 ,  0.06604467, -0.33224816,  0.40452626,  0.79497091,
          0.76190996,  0.10988872,  0.98480029,  0.41172975,  0.41553162,
         -0.21798775]]),
 array([[-0.3131676 ,  0.06604467, -0.33224816,  0.40452626,  0.79497091,
          0.76190996,  0.10988872,  0.98480029,  0.41172975,  0.41553162,
         -0.21798775]]))

In [25]:
regressor.encode(v)

array([[ 0.68196374, -0.74958611, -0.60189589, -0.45443407, -0.81894931,
        -1.04605621,  0.02830256, -0.61587576, -0.70138217]])